# Terminaison et correction des algorithmes (18 juin 2019)
## DIU 2019, Université de Bordeaux
### Cedric Chauve, cedric.chauve@u-bordeaux.fr

## Notes (cedric, 18/06/2019).
Il s'agit d'un brouillon, qui comprend un mélange du matériel présenté dans les transparents de Nicolas Nisse et des algorithmes vus dans le cours sur la complexité. Il faudrait ajouter des éléments sur les algorithmes gloutons et idéalement d'autres exemples pour éviter de revoir toujours les mêmes algorithmes. Le style est sans doute aussi trop formel.

Préambule.
Ce calepin Jupyter est le squelette du cours du Mercredi 26 juin portant sur les techniques de preuve de terminaison et correction des algorithmes. Il contient la plus grande partie du matériel présenté durant les trois heures du cours, introduit de manière relativement informelle et en utilisant plusieurs exemples relativement simple. Un traitement plus complet peut se trouver dans tout texte d'introduction aux algorithmes, par exemple les transparents du cours de Nicolas Nisse disponibles sur le repo github du DIU. 

Remarques.
Tous les algorithmes dont on analyse la complexité dans ce document sont implementés dans des cellules de type "code" permettant de les exécuter si nécessaire.

Pour tirer avantage du format offert par les calepins Jupyter, chaque section de ce document se termine par une cellule vide "Notes" permettant de prendre des notes en direct durant le cours.

## 1. Introduction

**Algorithme.** Séquence d’opérations (élémentaires) qui, étant données des entrées, calcule une sortie.

**Spécification.** L'entrée d'un algorithme doit satisfaire des *préconditions*. La sortie doit satisfaire des *postconditions*.

**Terminaison.** Un algorithme termine si il produit une sortie.

**Correction.** Un algorithme est correct si, appliqué à un entrée qui satisfait les préconditions, il termine et la sortie satisfait les postconditions.

### Exemple 1 : tri d'une permutation.

*Entrée.* Un tableau $T$ de $n$ éléments.  
*Sortie.* Le tableau $T$.     
*Préconditions.* Chaque entier de $\{1,\dots,n\}$ apparaît exactement une fois dans $T$.  
*Postconditions.* Pour toute position $i$ de $T$ sauf la dernière, $T[i]<T[i+1]$.


### Exemple 2 : tri d'un tableau d'entiers positifs.

*Entrée.* Un tableau $T$ de $n$ élément.  
*Sortie.* Le tableau $T$.  
*Préconditions.* Chaque élément de $T$ est un entier positif.  
*Postconditions.* Pour toute position $i$ de $T$ sauf la dernière, $T[i]\leq T[i+1]$.

### Exemple 3 : calcul de la somme des $n$ premiers entiers naturels.

*Entrée.* Un nombre $n$.   
*Sortie.* Un nombre $N$.  
*Préconditions.* $n$ est un entier positif.  
*Postconditions.* $N=1+2+\dots+n$.

### Exemple 4 : recherche des occurrences d'un motif dans une séquence d'ADN.

*Entrée.* Deux séquences $T$ et $M$ (le texte et le motif) de longueurs respectives $n$ et $m$.  
*Sortie.* Une liste $O = \{i_1,\dots,i_k\}$ d'entiers.  
*Préconditions.* $T$ et $N$ sont des séquences sur l'alphabet $\{A,C,G,T\}$.  
*Postconditions.* Pour $j=1,\dots,k$, $i_j+m-1 \leq n$ et $T[i_j\cdots i_j+m-1]=M$. Il n'existe pas $p \notin O$ tel que $T[p\cdots p+m-1]=M$.

### Notes.

## 2. Terminaison

Comment prouver la terminaison d'une suite d'instructions (i.e. un candidat algorithme)?

**Le cas facile.** Si la suite d'instructions n'est pas récursive et n'utilise pas de boucle, alors elle termine (en temps $\Theta(1)$).

**Le cas général.** Il faut définir un *convergent*, i.e. une quantité qui diminue à chaque itération/appel récursif, vivant dans un ensemble ordonné bien fondé (où il n’existe pas de suites infinies strictement décroissantes) et telle que lorsqu'elle a atteint sa valeur minimum, l'algorithme n'a plus qu'un nombre borné d'instructions à éxécuter.

### 2.1. Exemples de non-terminaison

Voici deux suites d'instructions prenant en entrée un entier positif $n$ et qui ne terminent pas.

In [1]:
# !! Ne pas exécuter ces suites d'instructions. Cela serait stupide.

def algo_stupide_1(n):
    i = n+1
    while i>n:
        i += 1
    print("Terminé")
    
def algo_stupide_2(n):
    i = n
    x = 1.0/float(n)
    while x>0:
        i += 1
        x += 1.0/float(i)
    print("Terminé")    

*algo_stupide_1* ne termine pas. La seule quantité modifiée par cette suite d'instructions est $i$ et elle ne fait que croître.

*algo_stupide_2* ne termine pas. La variable $x$ diminue à chaque itération mais vit dans un domaine qui n'est pas bien fondé (les nombres réels de l'intervalle $[0,1]$).

#### Notes.

### 2.2. Somme des premiers entiers naturels.

In [2]:
def somme_entiers_naturels_v1(n):
    somme = 0
    for i in range(1,n+1):
        somme += i
    return(somme)

def somme_entiers_naturels_v2(n):
    somme = n*(n+1)/2
    return(somme)

*somme_entiers_naturels_v2* ne contient ni boucle, ni appel récursif, donc termine.

*somme_entiers_naturels_v1* termine car le convergent $n-i$ vit dans le domaine bien fondé $\{1,2,\cdots,n\}$, décroît à chaque itération (il converge vers $0$) et lorsqu'il atteint $0$ (i.e. $i=n$) l'algorithme n'exécute qu'une instruction. 

De manière générale, un algorithme non-récursif n'utilisant que des boucles *for* tel que pour toute boucle, son indice n'est pas modifié à l'intérieur de la boucle termine.

#### Notes.

### 2.3. Le tri par insertion.

In [3]:
def tri_Insertion(T):
    n = len(T)
    for i in range(1,n):
        j = i-1;
        x = T[i]
        while j>=0 and x<T[j]:
            T[j+1] = T[j]
            j -= 1
        T[j+1] = x

On observe une boucle *for* dont l'indice $i$ n'est pas modifié, mais aussi une boucle *while*. 

Il faut donc trouver un convergent pour chaque itération de la boucle *for* qui nous assure que la boucle *while* termine : la variable $j$ décroît à chaque itération de la boucle *while* et vit dans le domaine $\{-1,0,\cdots,i-1\}$ qui est bien fondé. 

De plus, quand $j$ atteint sa valeur minimum (au pire $-1$), la boucle n'est suivie que d'une instruction.

Donc aucune exécution de la boucle *while* n'est infinie et l'algorithme termine.

#### Notes.

### 2.4. Le tri bulle.

Commençons par regarder la première version du tri bulle, dont nous avions vu qu'elle souffrait du défaut qu'elle prend un temps $\Theta(n^2)$ sour toute entrée $T$, même si $T$ est trié. 

In [4]:
def tri_Bulle_v1(T):
    n = len(T)
    for i in range(n-1,0,-1):
        for j in range(i):
            if T[j]>T[j+1]:
                aux    = T[j]
                T[j]   = T[j+1]
                T[j+1] = aux

On observe deux boucles *for*, dont les indices ne sont pas modifiés, donc termine. Formellement on devrait procéder en deux étapes :
- l'indice $i$ vit dans l'ensemble bien fondé $\{0,1,\cdots,n-2\}$ et décroît à chaque itération (donc $i$ est le convergent pour la boucle externe qui ne sera pas exécutée un nombre infini de fois),
- l'indice $j$ vit dans l'ensemble bien fondé  $\{0,1,\cdots,i-1\}$ et $i-j$ décroît à chaque itération.

Cet algorithme, bien que non satisfaisant, est cependant intéressant car il est facile d'en prouver la terminaison et la complexité.

Passons maintenant à la version plus classique, qui itère une boucle externe *while* tant que le tableau n'est pas trié.

In [5]:
def tri_Bulle_v2(T):
    n = len(T)
    trie = False
    while not trie:
        trie = True
        for j in range(n-1):
            if T[j]>T[j+1]:
                aux    = T[j]
                T[j]   = T[j+1]
                T[j+1] = aux
                trie = False

La terminaison de la boucle *for* est immédiate.

Le convergent pour cet algorithme n'est pas immédiat à trouver. Il ne peut pas s'agir d'une variable car on n'en introduit qu'une, $j$, qui ne décroît pas forcément lors de chaque itération de la boucle *while*. Il faut en trouver un qui se base non pas sur la structure de l'algorithme, mais sur le traitement effectué, à savoir le déplacement d'éléments de $T$.

Le convergent est ici le nombre de paires d'indices $\{a,b\}$ tels que $0\leq a<b< n$ et $T[a]>T[b]$, connu comme nombre d'*inversions* dans $T$. Ce nombre est un entier qui vit dans le domaine bien fondé $\{0,1,\cdots,n(n-1)/2\}$. Il peut donc servir de convergent et il faut donc prouver qu'il décroît strictement à chaque itération de la boucle *while*. Appelons le $I$.

*Cas 1.* Si $I=0$ au départ d'une itération de la boucle *while*, il n'existe pas d'indice $j$ tel que $T[j]>T[j+1]$ (sinon $I>0$) donc le booléen *trie* n'est pas modifié et il s'agit de la dernière itération de la boucle *while*.

*Cas 2.* Si $I>0$ au départ d'une itération de la boucle *while*, on veut montrer qu'il existe $j$ tel que $T[j]>T[j+1]$. Si cela n'est pas le cas, alors $T$ est trié et $I=0$, donc une contradiction. Donc un tel indice existe et $\{j,j+1\}$ forme une inversion. L'algorithme échange $T[j]$ et $T[j+1]$, donc cette inversion disparaît. Cet échange de deux éléments consécutifs ne crée pas de nouvelle inversion. Donc $I$ décroît.

On vient de démontrer que le nombre d'inversions est un convergent pour cet algorithme, qui termine.

#### Exercice.
Appliquer le même raisonnement, utilisant le même convergent, à la version suivante du tri bulle. Quelle est la complexité de cet algorithme dans le pire cas?

In [6]:
def tri_Bulle_v3(T):
    n = len(T)
    trie = False
    while not trie:
        trie = True
        j = 0
        while j<n-1 and T[j]<=T[j+1]:
            j += 1
        if j<n-1:
            aux    = T[j]
            T[j]   = T[j+1]
            T[j+1] = aux
            trie = False
            

#### Notes.

### 2.5. Le tri fusion.

Le tri fusion est intéressant car il s'agit d'un algorithme récursif. On va voir que cela simplifie la preuve de terminaison.

In [7]:
# Fusionner deux sous-tableaux triés T[g..m] et T[m+1..d] en un tableau T[g..d] trié
def fusionner(T,g,m,d):
    R = [0]*(d-g+1)
    i = g
    j = m+1
    k = 0
    while i<=m and j<=d:
        if T[i]<=T[j]:
            R[k] = T[i]
            i += 1
        else:
            R[k] = T[j]
            j += 1
        k += 1
    while i<=m:
        R[k] = T[i]
        i += 1
        k += 1
    while j<=d:
        R[k] = T[j]
        j += 1
        k += 1
    for k in range(len(R)):
        T[g+k] = R[k]
        
def tri_Fusion_Rec(T,g,d):
    if g<d:
        m = (g+d)//2
        nb_etapes += tri_Fusion_Rec(T,g,m)
        nb_etapes += tri_Fusion_Rec(T,m+1,d)
        nb_etapes += fusionner(T,g,m,d)

def tri_Fusion(T):
    n = len(T)
    return(tri_Fusion_Rec(T,0,n-1))

Le point important est de prouver d'abord que *fusionner* termine. La structure en 4 boucles simples rend la tâche aisée:
- la première boucle a pour convergent la paire $(m-i,d-j)$ qui vit dans le domaine bien fondé des paires d'entiers $(a,b)\in \{0,\cdots,m,m+1\}\times\{0,\cdots,d,d+1\}$ et où $(a,b)<(c,d)$ si $a<c$ ou $a=c,b<d$ (ordre lexicographique);
- la seconde boucle a pour convergent $m-i$ qui vit dans $\{0,\cdots,m,m+1\}$;
- la troisème boucle a pour convergent $d-j$ qui vit dans $\{0,\cdots,d,d+1\}$;
- la dernière boucle est une boucle *for* sans modification de l'indice $k$ dans la boucle.

On peut alors procéder par induction sur la taille du tableau $T$ à trier.

*Cas de base.* Supposons que $T$ est réduit à un élément, i.e. $g=d$. Alors *tri_Fusion_Rec* ne fait rien, donc termine.

*Hypothèse d'induction.* Pour $1\leq d-g+1<n$, *tri_Fusion_Rec(T,g,d)* termine.

*Cas général.* Soit $T[g..d]$ un tableau de taille $n$. Par induction, si $g<d$, *tri_Fusion_Rec(T,g,m)* et *tri_Fusion_Rec(T,m+1,d)* terminent. On a prouvé précédemment que *fusionner(T,g,m,d)* termine. Donc *tri_Fusion_Rec(T,g,d)* termine.

La preuve est simple, du fait de la structure récursive du programme qui ouvre la porte naturellement à une preuve par induction.

### 2.6. La suite de Syracuse.

On va maintenant étudier un algorithme pour lequel on ne sait pas prouver qu'il termine, bien qu'on le conjecture.

Soit $u_0$ un entier psoitif arbitraire. On définit la suite infinie d'entiers $u_0,u_1,u_2,\cdots$ comme suit:
$$u_{i+1} = u_i/2\mbox{ si $u_i$ est pair}$$
$$u_{i+1} = 3u_i+1\mbox{ si $u_i$ est impair}$$

On conjecture que pour toute valeur de $u_0$, il existe $n$ tel que $u_n=1$. On veut donc trouver la plus petite valeur de $n$ telle que $u_n=1$.

In [8]:
def syracuse(u_0):
    u_n = u0
    n = 0
    while u_n>1:
        if u_n % 2 == 0:
            u_n = u_n/2
        else:
            u_n = 3*u_n+1
        n += 1
    return(n)

On ne sait pas prouver que cet algorithme termine. Il est composé d'une boucle while itérant sur un indice qui peut diminuer ou croître dépendant de sa parité.

### 2.7. L'algorithme d'Euclide.

Cet algorithme calcule le *plus grand diviseur commun* (pgcd) de deux nombres entiers $a$ et $b$.

In [9]:
def euclide_pgcd(a,b):
    x = a
    y = b
    while y > 0:
        aux = y
        y = x % y
        x = aux
    return(x)

Cet algorithme a une seule boucle *while*. Le convergent est l'entier $y$ qui vit dans le domaine bien fondé $\{0,\cdots,y\}$. A chaque itération, la nouvelle valeur $y'$ de $y$ est le reste de la division de $x$ par $y$, donc $y'<y$. Donc $y$ décroît strictement, ce qui prouve la terminaison. 

## 3. Correction des algorithmes.

On veut maintenant prouver qu'un algorithme qui termine, si on l'applique à une entrée qui satisfait les préconditions, produit une sortie qui satisfait les postconditions.

### 3.1. Le tri bulle.

Prouver la correction de l'algorithme *tri_Bulle_v2* peut se faire en utilisant le convergent utilisé pour en prouver la terminaison.

Pour rappel, ce convergent, noté $I$, est défini comme le nombre de paires d'indices $\{a,b\}$ tels que $0\leq a<b< n$ et $T[a]>T[b]$. 

Il est clair que si $I=0$, le tableau $T$ est trié.

On a démontré que ce convergent décroît à chaque itération de la boucle *while*. Il faut donc montrer que quand l'algorithme se termine, $I=0$.

Dans notre preuve de terminaison, on a montré que $I>0$ implique que $I$ décroît. 

Donc $I$ converge vers $0$ et l'algorithme est correct.

### 3.2. Invariants et la somme des premiers entiers naturels.

Une autre technique de preuve de correction utilise la notion d'*invariant*. 

---
**Définition.**  Un invariant est un ensemble de propriétés qui relient les variables d'un algorithme et sont vraies (en général à des étapes spécifiées de l'algorithme). Les valeurs des variables peuvent changer mais pas les relations qui les lient.  

---
Un invariant peut servir à prouver la correction d'un algorithme si les propriétés qu'il contient sur la sortie implique qu'elle satisfait les postconditions définissant une sortie correcte.

Pour l'algorithme *somme_entiers_naturels_v1(n)*, on peut définir l'invariant suivant: après l'itération $i$ de la boucle $somme = 1+2+\cdots+i$.

On prouve généralement un invariant par induction.

*Cas de base*: pas d'itération. On a bien $somme=0$, donc l'invariant est vrai.

*Hypothèse d'induction.* Après l'itération $0\leq i-1$ de la boucle $somme = 1+2+\cdots+(i-1)$.

*Cas général.* Quelle est la valeur de $somme$ après l'itération $i$? Il s'agit de $somme+i=1+2+\cdots+(i-1)+i$ par induction.

### 3.3. L'algorithme d'Euclide.

Invariant : avant chaque itération de la boucle *while*, $pgcd(x,y)=pgcd(a,b)$.

Si cet invariant est satisfait, on a deux cas :
- On ne rentre pas dans la boucle, donc $b=0$, ce qui implique $x=a=pgcd(a,b)$ et on retourne bien $x=pgcd(a,b)$.
- On itère la boucle au moins une fois; la condition d'entrée dans la boucle étant $y=0$, quand l'algorithme détecte que $y=0$, on a $x=pgcd(x,0)$ (dernière instruction de la boucle), donc par l'invariant $x=pgcd(a,b)$ et on retourne $x$ qui satisfait la postcondition d'être le pgcd de $a$ et $b$.

Pour prouver cet invariant, on utilise l'induction sur le nombre d'itérations dela boucle.

*Cas de base.* Aucune itération. $y=b$. On a $x=a,y=b$ donc $pgcd(a,b)=pgcd(x,y)$.

*Hypothèse d'induction.$ Après $i-1\geq 0$ itératons de la boucle, $pgcd(a,b)=pgcd(x,y)$.

*Cas général.* Après $i$ itérations. 
Durant la  $i^{eme}$ itération, soient $X$ et $Y$ les valeurs de $x$ et $y$ au début de l'itération. 
On calcule les nouvelles valeurs de $x$ et $y$ par $y = X\mod Y$ et $x=Y$. 
On a donc $X = kY + y$, donc, par le théorème fondamental, $pgcd(X,Y)=pgcd(Y,x)=pgcd(x,y)$. 
Comme par induction on a $pgcd(a,b)=pgcd(X,Y)$, on a $pgcd(a,b)=pgcd(x,y)$. 
Donc avant l'itération suivante, $pgcd(a,b)=pgcd(x,y)$ et l'invariant est satisfait.

### 3.4. Le tri fusion.

La preuve de correction de cet algorithme se fait en deux temps : prouver que *fusionner* est correcte, puis prouver que *tri_Fusion_Rec* est correcte.

La précondition de *fusionner(T,g,m,d)* est que $T[g..m]$ et $T[m+1..d]$ sont triés.  
La postcondition est que $T[g..d]$ est trié.  
L'analyse de la dernière boucle (qui recopie $R$ dans $T[g..d]$) implique que cette postcondition est équivalente au fait que le tableau $R$ contient les éléments de $T[g..m]$ et $T[m+1..d]$ triés.

On remarque que l'algorithme *fusionner* incrémente à chaque itération des trois premières boucles. 

On peut utiliser $k$ pour énoncer l'invariant suivant: juste après chaque affectation de $k$, $R[0..k-1]$ contient les $k$ plus petits éléments de $T[g..m]$ et $T[m+1..d]$ triés en ordre croissant.

Là encore on peut prouver cet invariant par induction sur $k$.

**A finir.**